## Benchmark Optimization Problems

This notebook compares `hetGPys` performance on a test suite of optimization problems to `hetGP` and `GPyTorch`

Datasets were chosen from Picheny et. al (2012): https://link.springer.com/article/10.1007/s00158-013-0919-4#Sec16

And R implementations from: https://www.sfu.ca/~ssurjano/optimization.html

See `R/generate_data.R` for the implementations. For ease of design, all problems were evaluted on a unit hypercube $[0,1]^d$ where $d$ is the dimension of the dataset
* Branin
* Goldstein-Price
* Rosenbrock
* Hartmann (4D)
* Hartmann (6D)
* Sphere



Experiments were run on the Gaussian and Matern 5/2 kernels. The unique design locations of $N=500$ was chosen arbitrarily.

### Experiments with `hetgpy`
* See `py/run_homGP.py` for the implementation

### Experiments with `GPyTorch` (naive case)

* See `py/run_GPyTorch_with_botorch_naive.py` for the implementation

### Experiments with `GPyTorch` (Stochastic Kriging case)
* See `py/run_GPyTorch_with_botorch_SK.py` for the implementation

In [1]:
import pandas as pd
import numpy as np
GPyTorch_rep_tests  = pd.read_csv('py/hetgpy-hom-GPyTorch-tests-SK.csv')
py_tests            = pd.read_csv('py/hetgpy-hom-GP-tests.csv')
torch_tests         = pd.read_csv('py/hetgpy-hom-GPyTorch-tests-naive.csv')

GPyTorch_rep_tests['experiment'] = '02-GPyTorch (SK)'
py_tests['experiment']           = '03-hetGPy'
torch_tests['experiment']        = '01-GPyTorch (naive)'

py_tests['theta'] = py_tests['theta'].str.replace('[','').str.replace(']','')
cols = ['covtype','name','experiment','time','ll','nfev','theta','msg']



df = pd.concat([
    GPyTorch_rep_tests[cols],
    py_tests[cols],
    torch_tests[cols]
]).sort_values(['covtype','name'])
df.loc[df['covtype']=='Matern5_2','covtype'] = 'Matern $\\nu = 5/2$'
df = df.set_index(['covtype','name'])
df['msg'] = True
df['time'] = np.round(df['time'].values,3)

In [2]:
str_sum = lambda s: ''.join(s)
numerics = df.reset_index().pivot_table(index= ['covtype','name'], columns = 'experiment', values = ['ll','msg','time','nfev'])
col_mapper = lambda x: x[3:]
table = (numerics[['time','nfev']]
         .rename(mapper = col_mapper,level=1,axis='columns')
         )

# format
table['time'] = table['time'].astype(str)
table['nfev'] = table['nfev'].astype(int)
# format
print(table.to_latex())

\begin{tabular}{lllllrrr}
\toprule
 &  & \multicolumn{3}{r}{time} & \multicolumn{3}{r}{nfev} \\
 & experiment & GPyTorch (naive) & GPyTorch (SK) & hetGPy & GPyTorch (naive) & GPyTorch (SK) & hetGPy \\
covtype & name &  &  &  &  &  &  \\
\midrule
\multirow[t]{5}{*}{Gaussian} & Branin & 98.449 & 5.622 & 2.693 & 13 & 18 & 13 \\
 & Goldstein-Price & 119.019 & 3.688 & 1.895 & 21 & 12 & 10 \\
 & Hartmann-4D & 98.566 & 4.578 & 4.936 & 18 & 15 & 16 \\
 & Hartmann-6D & 466.381 & 19.845 & 21.472 & 95 & 69 & 40 \\
 & Sphere-6D & 206.05 & 13.989 & 7.702 & 42 & 51 & 20 \\
\cline{1-8}
\multirow[t]{5}{*}{Matern $\nu = 5/2$} & Branin & 90.501 & 4.765 & 6.563 & 13 & 12 & 20 \\
 & Goldstein-Price & 153.033 & 4.406 & 4.005 & 20 & 11 & 15 \\
 & Hartmann-4D & 201.208 & 6.895 & 6.755 & 21 & 18 & 16 \\
 & Hartmann-6D & 339.387 & 28.427 & 45.23 & 53 & 67 & 57 \\
 & Sphere-6D & 147.118 & 16.034 & 16.345 & 24 & 38 & 14 \\
\cline{1-8}
\bottomrule
\end{tabular}



In [3]:
numerics[['time','nfev']].rename(mapper = col_mapper,level=1,axis='columns')

time                        \
experiment                         GPyTorch (naive) GPyTorch (SK)  hetGPy   
covtype            name                                                     
Gaussian           Branin                    98.449         5.622   2.693   
                   Goldstein-Price          119.019         3.688   1.895   
                   Hartmann-4D               98.566         4.578   4.936   
                   Hartmann-6D              466.381        19.845  21.472   
                   Sphere-6D                206.050        13.989   7.702   
Matern $\nu = 5/2$ Branin                    90.501         4.765   6.563   
                   Goldstein-Price          153.033         4.406   4.005   
                   Hartmann-4D              201.208         6.895   6.755   
                   Hartmann-6D              339.387        28.427  45.230   
                   Sphere-6D                147.118        16.034  16.345   

                                               nfev                       
experiment                         GPyTorch (naive) GPyTorch (SK) hetGPy  
covtype            name                                                   
Gaussian           Branin                      13.0          18.0   13.0  
                   Goldstein-Price             21.0          12.0   10.0  
                   Hartmann-4D                 18.0          15.0   16.0  
                   Hartmann-6D                 95.0          69.0   40.0  
                   Sphere-6D                   42.0          51.0   20.0  
Matern $\nu = 5/2$ Branin                      13.0          12.0   20.0  
                   Goldstein-Price             20.0          11.0   15.0  
                   Hartmann-4D                 21.0          18.0   16.0  
                   Hartmann-6D                 53.0          67.0   57.0  
                   Sphere-6D                   24.0          38.0   14.0